In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_ETH = "ETHUSDT"
BINANCE_XRP = "XRPUSDC"
BINANCE_SOL = "SOLUSDC"

BINANCE_SYMBOL = BINANCE_ETH
USD_QUOTES = ["SOL", "XRP", "MATIC"]
quote_in_usd = len([prefix for prefix in USD_QUOTES if BINANCE_SYMBOL.upper().startswith(prefix)]) > 0

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [5]:
target_period_hours = 1 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Tue 09 Apr, 08:00 (23 hours left)
current price: 3597.42
-------------------------------
hit ratio: 27%
cost: 0.032 / value: 0.023, gain% = -0.81%
($) cost: 113.32 / value: 84.20, average gain = -29.12
buy put 3575.0
buy call 3625.0
-------------------------------
hit ratio: 26%
cost: 0.026 / value: 0.019, gain% = -0.69%
($) cost: 91.73 / value: 66.95, average gain = -24.79
buy put 3550.0
buy call 3650.0
-------------------------------
hit ratio: 23%
cost: 0.019 / value: 0.013, gain% = -0.63%
($) cost: 70.15 / value: 47.53, average gain = -22.62
buy put 3525.0
buy call 3700.0
-------------------------------
hit ratio: 20%
cost: 0.013 / value: 0.009, gain% = -0.40%
($) cost: 48.57 / value: 34.09, average gain = -14.48
buy put 3500.0
buy call 3750.0


In [15]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
3500.0,153.112809,NaN,0.0405,0.0465,33.188285,0.009209,0.0115,0.0125
3525.0,134.176139,NaN,0.0370,0.0405,39.251615,0.010891,0.0140,0.0150
3550.0,116.447980,NaN,0.0335,0.0360,46.523456,0.012909,0.0170,0.0175
3575.0,100.183013,NaN,0.0300,0.0325,55.258489,0.015333,0.0200,0.0210
3600.0,85.744017,NaN,0.0270,0.0280,65.819493,0.018263,0.0235,0.0250
3625.0,73.403827,0.020368,0.0235,0.0250,78.479303,NaN,0.0275,0.0285
3650.0,62.869078,0.017445,0.0210,0.0220,92.944554,NaN,NaN,NaN
3675.0,53.814820,0.014932,0.0185,0.0195,108.890296,NaN,0.0005,NaN
3700.0,45.988805,0.012761,0.0165,0.0175,126.064281,NaN,0.0005,NaN


# Sanity checks

In [8]:
size = 6
option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size=size)
put_weights = [0.] * (2 * size + 1)
call_weights = [0.] * (2 * size + 1)
put_weights[size - 5] = 1.
put_weights[size - 2] = -1.
call_weights[size + 2] = -1.
call_weights[size + 5] = 1.

simulation = trading_model.simulate_strategy(option_chain_df, put_weights, call_weights, quote_in_usd=quote_in_usd)
print(simulation)

target expiry: Tue 09 Apr, 08:00 (23 hours left)
current price: 3597.42
-------------------------------
hit ratio: 61%
cost: -0.013 / value: -0.012, gain% = 0.16%
($) cost: -47.85 / value: -42.23, average gain = 5.62
buy put 3475.0
sell put 3550.0
sell call 3650.0
buy call 3800.0
